In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
books = pd.read_csv('data/BX-Books.csv', sep=";", on_bad_lines='skip', encoding='latin-1', low_memory=False)

In [3]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]


In [4]:
books.rename(columns={"Book-Title":"b_title",
                         "Book-Author": "author",
                    "Year-Of-Publication":"year",
                      "Publisher":"publisher",
                  "Image-URL-L":"img_ul"}, inplace = True)

In [5]:
users  =pd.read_csv('data/BX-users.csv', sep=";", on_bad_lines='skip', encoding='latin-1', low_memory=False)

In [6]:
ratings=pd.read_csv('data/BX-Book-ratings.csv', sep=";", on_bad_lines='skip', encoding='latin-1', low_memory=False)

In [7]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


In [8]:
ratings.rename(columns={
   "User-ID":"uid",
   "Book-Rating":"rates"},inplace= True)

In [9]:
users.rename(columns={
       "User-ID":"uid"},inplace= True)

In [10]:
ratings['uid'].value_counts()

uid
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [11]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [12]:
n_rating = ratings_with_name.groupby('b_title').count()['rates'].reset_index()
n_rating.rename(columns ={'rates':'v_ratings'},inplace = True)
n_rating

,b_title,v_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [13]:
print(ratings_with_name['rates'].dtype)

int64


In [14]:
ratings_with_name['rates'] = pd.to_numeric(ratings_with_name['rates'], errors='coerce')

In [15]:
avg_rating = ratings_with_name.groupby('b_title')['rates'].mean().reset_index()
avg_rating.rename(columns={"rates":"avg_ratings"},inplace= True)
avg_rating


,b_title,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [16]:
popularity = n_rating.merge(avg_rating, on ="b_title")
popularity.head

<bound method NDFrame.head of                                                   b_title  v_ratings  \
0        A Light in the Storm: The Civil War Diary of ...          4   
1                                   Always Have Popsicles          1   
2                    Apple Magic (The Collector's series)          1   
3        Ask Lily (Young Women of Faith: Lily Series, ...          1   
4        Beyond IBM: Leadership Marketing and Finance ...          1   
...                                                   ...        ...   
241066                                      Ã?Â?lpiraten.          2   
241067                     Ã?Â?rger mit Produkt X. Roman.          4   
241068                                Ã?Â?sterlich leben.          1   
241069                              Ã?Â?stlich der Berge.          3   
241070                                  Ã?Â?thique en toc          2   

        avg_ratings  
0          2.250000  
1          0.000000  
2          0.000000  
3          8.0000

In [17]:
popularity=popularity[popularity['v_ratings']>=250].sort_values('avg_ratings',ascending=False).head(50)

In [18]:
popularity=popularity.merge(books, on ="b_title").drop_duplicates('b_title')[['b_title','author','img_ul','v_ratings','avg_ratings']]

In [19]:
popularity.head()

,b_title,author,img_ul,v_ratings,avg_ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453


In [20]:
import pickle 
pickle.dump(popularity, open('popularity.pkl','wb'))